# 01. JSON 데이터

---

## 1) JSON 소개

---

- JSON은 경량의 **데이터 표현 형식**.
- 웹, 모바일 등을 중심으로 **서로 다른 플랫폼간의 데이터 교환**을 위하여 활용되고 있는 사실상의 산업 표준.
- OpenAPI등을 통해 다른 곳에서 제공하는 데이터를 수집할 경우 JSON 형식으로 되어 있는 경우가 많음.
- Python에서도 HTTP 통신을 기반으로 OpenAPI로부터 JSON 형식의 데이터를 수집하고 이를 데이터 프레임으로 변환하는 형태의 방식을 많이 활용함.

## 2) JSON 구조의 이해

---

### 기본 형식

- key와 value의 쌍으로 이루어진 데이터 표현 형식으로 **Python의 딕셔너리와 동일한 구조**임
- 값에는 숫자, 문자열, 논리값(true/false) 모두 가능하며 값이 문자열인 경우는 쌍따옴표나 홑따옴표로 감싸서 표현
- 논리값의 경우 파이썬은 `True` / `False` 로 표기하지만 JSON은 `true` / `false` 로 표기한다. 
(첫 글자 대문자 여부)
- JSON 상에서 결측치는 `null` 로 표기한다. (따옴표 없음. 모두 소문자)

In [2]:
# JSON 가져오기

import requests
import json
from pandas import DataFrame
from matplotlib import pyplot as plt

# JSON 데이터 URL 지정
simple_json_url = "http://www.itpaper.co.kr/data/simple.json"

# 브라우저 버전 정보
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"

# HTTP 헤더 정보 구성
header_info = {'User-agent': user_agent, 'referer': None}

# 데이터 가져오기
r = requests.get(simple_json_url, headers=header_info)

# 결과 검사
if r.status_code != 200:
    # 에러코드와 에러메시지를 문자열로 구성
    err_msg = "%d %s 에러가 발생했습니다." % (r.status_code, r.reason)
    # 강제로 에러를 발생시킨다.
    raise Exception(err_msg)

r.encoding = 'utf-8'

r.text

'{\n    "name": "갤럭시 S6",\n    "type": "삼성",\n    "img": "http://itpaper.co.kr/demo/app/img/GalaxyS6.png"\n}'

In [3]:
# JSON 형식의 문자열을 딕셔너리로 변환
result = json.loads(r.text)
result

{'name': '갤럭시 S6',
 'type': '삼성',
 'img': 'http://itpaper.co.kr/demo/app/img/GalaxyS6.png'}

### 딕셔너리를 DataFrame으로 변환하기

DataFrame을 생성이 가능한 파라미터 형식 

1. 2차 리스트
2. 모든 원소가 같은 크기의 리스트인 딕셔너리

    ```python
    {"a": [1, 2, 3, 4], "b": [10, 20, 30, 40] }
    ```

3. 모든 원소가 같은 구조의 딕셔너리인 리스트

    ```python
    [ {"a": 1, "b": 2}, {"a": 10, "b": 20}, {"a": 100, "b": 200}]
    ```

In [4]:
data_list = [result]
simple_df = DataFrame(data_list)
simple_df

,name,type,img
0,갤럭시 S6,삼성,http://itpaper.co.kr/demo/app/img/GalaxyS6.png


## 리스트 형식의 데이터를 포함하는 JSON

---

### 데이터 수집

In [5]:
# JSON URL
json_list_url = "http://www.itpaper.co.kr/data/grade_card.json"

# 준비된 URL의 컨텐츠 가져오기
r = requests.get(json_list_url, headers=header_info)

# 접속에 실패한 경우에 대한 예외처리
if r.status_code != 200:
    # 에러코드와 에러메시지를 문자열로 구성
    err_msg = "%d %s 에러가 발생했습니다." % (r.status_code, r.reason)
    # 강제로 에러를 발생시킨다.
    raise Exception(err_msg)

# 인코딩 지정
r.encoding = "utf-8"

# 가져온 결과를 딕셔너리로 변환
result = json.loads(r.text)
result

{'grade_card': [{'이름': '철수',
   '학년': 1,
   '성별': '남자',
   '국어': 98,
   '영어': None,
   '수학': 88,
   '과학': 64},
  {'이름': '영희', '학년': 2, '성별': '여자', '국어': 88, '영어': 90, '수학': 62, '과학': 72},
  {'이름': '민수',
   '학년': 1,
   '성별': '남자',
   '국어': 92,
   '영어': 70,
   '수학': None,
   '과학': None},
  {'이름': '수현', '학년': 3, '성별': '여자', '국어': 63, '영어': 60, '수학': 31, '과학': 70},
  {'이름': '호영',
   '학년': 4,
   '성별': '남자',
   '국어': 120,
   '영어': 50,
   '수학': None,
   '과학': 88}]}

In [6]:
# student키에 해당하는 리스트를 추출하여 데이터프레임으로 변환한다.

student_df = DataFrame(result['grade_card'])
student_df

,이름,학년,성별,국어,영어,수학,과학
0,철수,1,남자,98,NaN,88.0,64.0
1,영희,2,여자,88,90.0,62.0,72.0
2,민수,1,남자,92,70.0,NaN,NaN
3,수현,3,여자,63,60.0,31.0,70.0
4,호영,4,남자,120,50.0,NaN,88.0


In [7]:
student_df = student_df.set_index('이름')
student_df

,학년,성별,국어,영어,수학,과학
이름,,,,,,
철수,1,남자,98,NaN,88.0,64.0
영희,2,여자,88,90.0,62.0,72.0
민수,1,남자,92,70.0,NaN,NaN
수현,3,여자,63,60.0,31.0,70.0
호영,4,남자,120,50.0,NaN,88.0


In [8]:
student_df2 = student_df.copy()

student_df2['평균'] = student_df.mean(axis=1)
student_df2

,학년,성별,국어,영어,수학,과학,평균
이름,,,,,,,
철수,1,남자,98,NaN,88.0,64.0,62.750000
영희,2,여자,88,90.0,62.0,72.0,62.800000
민수,1,남자,92,70.0,NaN,NaN,54.333333
수현,3,여자,63,60.0,31.0,70.0,45.400000
호영,4,남자,120,50.0,NaN,88.0,65.500000
